In [1]:
from IPython.display import display, Math, Markdown, HTML

from math import *
import numpy as np
import pandas as pd
import scipy.stats
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.linestyle"] = "dotted"
plt.rcParams["errorbar.capsize"] = 2
plt.rcParams["savefig.dpi"] = 200
# plt.rcParams["savefig.bbox"] = "tight"
# plt.rcParams["savefig.pad_inches"] = 0.01
plt.rcParams["savefig.format"] = "pdf"

# from uncertainties import unumpy as unp
from uncertainties import ufloat
# from uncertainties.unumpy import uarray
from uncertainties.unumpy import nominal_values as unominal
from uncertainties.unumpy import std_devs as ustdev

def display_ufloat(uf, var=None, dim=None):
    res = ''
    if var is not None:
        res += var + '='
    res += f"{uf:L}"
    if dim is not None:
        res += ' \\ ' + dim
    display(Math(res))

<center><b>I. Измерение петли гистерезиса<b><center>

In [2]:
df = pd.DataFrame(columns=["I", "K_X", "K_Y", "N_0", "N_int", "S", "2piR"])
#                           мА  В/дел  В/дел   ед      ед   см^2   см

df.loc[0] = [216.2, 0.05, 0.1, 35, 220, 3.8, 24]
df.loc[1] = [175.6, 0.02, 0.02, 40, 400, 3.0, 25]
df.loc[2] = [917.2, 0.1, 0.1, 40, 400, 1.2, 10]

df

,I,K_X,K_Y,N_0,N_int,S,2piR
0,216.2,0.05,0.10,35.0,220.0,3.8,24.0
1,175.6,0.02,0.02,40.0,400.0,3.0,25.0
2,917.2,0.10,0.10,40.0,400.0,1.2,10.0


<p float="left">
  <img src="images/img1.jpg" width="475" />
  <img src="images/img2.jpg" width="475" /> 
  <img src="images/img3.jpg" width="475" />
</p>

In [3]:
df["2X_s"] = 2 * np.array([2.6, 4.4, 3.5])
df["2Y_s"] = 2 * np.array([2.2, 2.1, 2.0])
df["2X_c"] = 2 * np.array([1.2, 0.6, 0.6])
df["2Y_r"] = 2 * np.array([2.1, 0.9, 0.9])

$$
H = \frac{I N_0}{2 \pi R}, \ I = K_X / R_0
$$

In [4]:
R_0 = 0.3  # Ом
I = df["K_X"] / R_0  # А/дел
df["H"] = I * df["N_0"] / (df["2piR"] / 100)  # (А/м)/дел

$$
B = \frac{R_и C_и}{S N_и}U_{вых}, \ U_{вых} = K_Y
$$

In [5]:
R_int = 20  # кОм
C_int = 20  # мкФ
df["B"] = (R_int * C_int / 1e3) / ((df["S"] / 1e4) * df["N_int"]) * df["K_X"]  # (с/м^2*В)/дел = Тл/дел

In [6]:
df

,I,K_X,K_Y,N_0,N_int,S,2piR,2X_s,2Y_s,2X_c,2Y_r,H,B
0,216.2,0.05,0.10,35.0,220.0,3.8,24.0,5.2,4.4,2.4,4.2,24.305556,0.239234
1,175.6,0.02,0.02,40.0,400.0,3.0,25.0,8.8,4.2,1.2,1.8,10.666667,0.066667
2,917.2,0.10,0.10,40.0,400.0,1.2,10.0,7.0,4.0,1.2,1.8,133.333333,0.833333


<center><b>II. Проверка калибровки осциллографа<b><center>

$$
K_X = 2R_0\sqrt{2}I_{ЭФ}/(2x) \ \ \ \ \ \ [В/дел]
$$

In [7]:
x = 5  # дел
I_ef = 1.135  # А
K_x = 2 * R_0 * 2 ** 0.5 * I_ef / (2 * x)  # В/дел
0.5, K_x  # В/дел

(0.5, 0.09630794359760778)

In [8]:
x = 5  # дел
I_ef = 0.577  # А
K_x = 2 * R_0 * 2 ** 0.5 * I_ef / (2 * x)  # В/дел
0.05, K_x  # В/дел

(0.05, 0.048960073529356544)

In [9]:
x = 5  # дел
I_ef = 0.224  # А
K_x = 2 * R_0 * 2 ** 0.5 * I_ef / (2 * x)  # В/дел
0.02, K_x  # В/дел

(0.02, 0.0190070302782944)

$$
K_Y = 2\sqrt{2}U_{ЭФ}/(2y) \ \ \ \ \ \ [В/дел]
$$

In [10]:
y = 2
U_ef = 0.131  # В
K_y = 2 ** 1.5 * U_ef / (2 * y)
0.1, K_y  # В/дел

(0.1, 0.09263098833543773)

<center><b>III. Определение &#964 — постоянной времени интегрирующей ячейки<b><center>

$$
\tau = RC = U_{ВХ}/(\omega U_{ВЫХ})
$$

In [11]:
tau = R_int * C_int / 1000 # к * кОм * мкФ = м * В/А * Кл/В = к * м * с = c

tau

0.4

In [12]:
U_in = 3.93  # В
U_out = 0.029  # В
omega = 2 * pi * 50  # рад/с

U_in / (omega * U_out)  # с

0.4313647767938957

$$
R \gg 1 / (\omega C)
$$

In [13]:
R_int, 1 / (omega * C_int)

(20, 0.00015915494309189532)

<center><b>IV. Обработка результатов<b><center>

In [14]:
df["H_max"] = df["2X_s"] / 2 * df["H"]  # А/м
df["B_s"] = df["2Y_s"] / 2 * df["B"]  # Тл
df["H_c"] = df["2X_c"] / 2 * df["H"]  # А/м
df["B_r"] = df["2Y_r"] / 2 * df["B"]  # Тл

df

,I,K_X,K_Y,N_0,N_int,S,2piR,2X_s,2Y_s,2X_c,2Y_r,H,B,H_max,B_s,H_c,B_r
0,216.2,0.05,0.10,35.0,220.0,3.8,24.0,5.2,4.4,2.4,4.2,24.305556,0.239234,63.194444,0.526316,29.166667,0.502392
1,175.6,0.02,0.02,40.0,400.0,3.0,25.0,8.8,4.2,1.2,1.8,10.666667,0.066667,46.933333,0.140000,6.400000,0.060000
2,917.2,0.10,0.10,40.0,400.0,1.2,10.0,7.0,4.0,1.2,1.8,133.333333,0.833333,466.666667,1.666667,80.000000,0.750000


<p float="left">
  <img src="images/img1.jpg" width="475" />
  <img src="images/img2.jpg" width="475" /> 
  <img src="images/img3.jpg" width="475" />
</p>

In [15]:
df["dY/dX_нач"] = [3.3/6.3, 1.3/3.2, 2.7/2.6]
df["dY/dX_max"] = [(12.5-9.2)/(9.2-8.0), (8.8-4.8)/(10.9-7.1), (4.7-2.7)/(3.8-2.6)]

mu0 = 1.25663706212e-6  # Гн/м
df["mu_нач"] = df["dY/dX_нач"] * df["B"] / df["H"] / mu0  # безразмерная величина
df["mu_max"] = df["dY/dX_max"] * df["B"] / df["H"] / mu0  # безразмерная величина

df

,I,K_X,K_Y,N_0,N_int,S,2piR,2X_s,2Y_s,2X_c,...,H,B,H_max,B_s,H_c,B_r,dY/dX_нач,dY/dX_max,mu_нач,mu_max
0,216.2,0.05,0.10,35.0,220.0,3.8,24.0,5.2,4.4,2.4,...,24.305556,0.239234,63.194444,0.526316,29.166667,0.502392,0.523810,2.750000,4102.812709,21539.766723
1,175.6,0.02,0.02,40.0,400.0,3.0,25.0,8.8,4.2,1.2,...,10.666667,0.066667,46.933333,0.140000,6.400000,0.060000,0.406250,1.052632,2020.521737,5235.359967
2,917.2,0.10,0.10,40.0,400.0,1.2,10.0,7.0,4.0,1.2,...,133.333333,0.833333,466.666667,1.666667,80.000000,0.750000,1.038462,1.666667,5164.883968,8289.319948


In [16]:
headers = ["Пермаллой (Fe-Ni)", "Кремнистое железо (Fe-Si)", "Феррит"]
rows = ["H_c", "B_s", "mu_нач", "mu_max"]
row_headers = ["$H_c$, $А/м$", "$B_s$, $Тл$", "$\mu_{нач}$", r"$\mu_{\text{max}}$"]
# display(Markdown(df[rows].T[[0, 2, 1]].to_markdown(headers=headers, index=row_headers)))

table = r'''
<div style="display: flex; justify-content: space-around;">

<div style="width: 45%;">
<h3>Экспериментальные данные</h3>

''' + df[rows].T[[0, 2, 1]].to_markdown(headers=headers, index=row_headers) + '''

</div>

<div style="width: 45%;">
<h3>Таблицчные данные</h3>

|                    |   Пермаллой (Fe-Ni)            |   Кремнистое железо (Fe-Si) |   Феррит             |
|:-------------------|-------------------------------:|----------------------------:|---------------------:|
| $H_c$, $А/м$       |           1–5                  |                20–100       |    10–100            |
| $B_s$, $Тл$        |           0.6–1.0              |                1.5–2.0      |    0.3–0.5           |
| $\\mu_{нач}$        |      10,000–100,000            |            4,000–20,000     |  1,000–10,000        |
| $\\mu_{\\text{max}}$ |     100,000–1,000,000          |           30,000–100,000    |  5,000–15,000        |

</div>

</div>

'''

display(Markdown(table))


<div style="display: flex; justify-content: space-around;">

<div style="width: 45%;">
<h3>Экспериментальные данные</h3>

|                    |   Пермаллой (Fe-Ni) |   Кремнистое железо (Fe-Si) |   Феррит |
|:-------------------|--------------------:|----------------------------:|---------:|
| $H_c$, $А/м$       |           29.1667   |                    80       |     6.4  |
| $B_s$, $Тл$        |            0.526316 |                     1.66667 |     0.14 |
| $\mu_{нач}$        |         4102.81     |                  5164.88    |  2020.52 |
| $\mu_{\text{max}}$ |        21539.8      |                  8289.32    |  5235.36 |

</div>

<div style="width: 45%;">
<h3>Таблицчные данные</h3>

|                    |   Пермаллой (Fe-Ni)            |   Кремнистое железо (Fe-Si) |   Феррит             |
|:-------------------|-------------------------------:|----------------------------:|---------------------:|
| $H_c$, $А/м$       |           1–5                  |                20–100       |    10–100            |
| $B_s$, $Тл$        |           0.6–1.0              |                1.5–2.0      |    0.3–0.5           |
| $\mu_{нач}$        |      10,000–100,000            |            4,000–20,000     |  1,000–10,000        |
| $\mu_{\text{max}}$ |     100,000–1,000,000          |           30,000–100,000    |  5,000–15,000        |

</div>

</div>

